# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [5]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [6]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [7]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["kmeans"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]

In [8]:
n_components = 20

In [9]:
def getKMeans(vectors, n_components, seed, path):
    model = KMeans(n_clusters=n_components, random_state=seed)
    model.fit(vectors)
    # save model
    os.makedirs(os.path.dirname(path), exist_ok=True)
    pickle.dump(model, open(path, "wb"))
    pred = model.predict(vectors)
    return pred

In [10]:
vectors_path = "../../data/20News/doc2vec/vector/"
models_path = "../../data/20News/doc2vec/KMeans/model/"
pred_path = "../../data/20News/doc2vec/KMeans/pred/"
for vector_dim in tqdm(vector_dims):
    for model_num in range(model_nums):
        vector_path = f"{vectors_path}{vector_dim}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")

        os.makedirs(f"{models_path}{vector_dim}", exist_ok=True)
        pred = getKMeans(
            vectors,
            seed=model_num,
            n_components=n_components,
            path=f"{models_path}{vector_dim}/{model_num}.sav",
        )

        # save prediction
        os.makedirs(f"{pred_path}{vector_dim}", exist_ok=True)
        with open(f"{pred_path}{vector_dim}/{model_num}.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(pred)

  0%|          | 0/14 [00:00<?, ?it/s]


FileNotFoundError: ../../data/20News/doc2vec/vector/[2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]/normalized/0.csv not found.